In [ ]:
%pip install pandas
%pip install folium
%pip install mapclassify
%pip install geopandas
%pip install selenium

In [3]:
import pandas as pd
import geopandas as gpd
import folium
import os
from IPython.display import display

In [4]:
# Load election data
election_data = pd.read_csv("data/election_results/kombinirani_rezultati.csv")

# Load GeoJSON data
geojson_data = gpd.read_file("data/administrative_units/gradovi_opcine_zupanije.geojson")

In [ ]:
print(election_data.head())
print(geojson_data.head())

# Ensure that the key columns are named consistently
election_data['Županija'] = election_data['Županija'].str.strip()  # Strip any extra spaces
election_data['Grad/općina'] = election_data['Grad/općina/država'].str.strip()

geojson_data['Županija'] = geojson_data['text_right'].str.strip()
geojson_data['Grad/općina'] = geojson_data['text_left'].str.strip()

parlamentarni_izbori = geojson_data.merge(election_data, on=['Županija', 'Grad/općina'], how='inner')

In [35]:
def create_interactive_election_map(parlamentarni_izbori):
    # Calculate the percentage of Važeći listići divided by Ukupno birača
    parlamentarni_izbori['percentage'] = (
        parlamentarni_izbori['Važeći listići'] / parlamentarni_izbori['Ukupno birača']
    ) * 100

    min_value = parlamentarni_izbori['percentage'].min()
    max_value = parlamentarni_izbori['percentage'].max()

    # Create a color map for the percentage (from blue to yellow to red)
    colormap = folium.LinearColormap(
        colors=['blue', 'yellow', 'red'],
        vmin=min_value,  # Minimum value for color map
        vmax=max_value,  # Maximum value for color map
        caption='Percentage of Važeći listići to Ukupno birača'
    )

    # Add color column based on percentage
    parlamentarni_izbori['color'] = parlamentarni_izbori['percentage'].apply(lambda x: colormap(x))

    # Convert Timestamp columns to strings
    for col in parlamentarni_izbori.select_dtypes(include=['datetime64[ns, UTC]', 'datetime64[ns]']).columns:
        parlamentarni_izbori[col] = parlamentarni_izbori[col].astype(str)

    # Prepare hover information with sorted percentage columns
    def create_hover_info(row):
        percentage_columns = [col for col in parlamentarni_izbori.columns if col.endswith('%')]
        sorted_percentages = sorted(
            [(col, row[col]) for col in percentage_columns if not pd.isna(row[col])],
            key=lambda x: x[1], reverse=True
        )

        hover_text = (
            f"Županija: {row['Županija']}<br>"
            f"Grad/Općina: {row['Grad/općina']}<br>"
            f"Ukupno birača: {row['Ukupno birača']}<br>"
            f"Važeći listići: {row['Važeći listići']}<br>"
            f"Percentage: {row['percentage']:.2f}%"
        )
        hover_text += ''.join([f"<br>{col}: {value}" for col, value in sorted_percentages])
        return hover_text

    parlamentarni_izbori['hover_info'] = parlamentarni_izbori.apply(create_hover_info, axis=1)

    # Create a folium map centered on Croatia
    m = folium.Map(location=[45.1, 15.2], zoom_start=7)

    # Function to style each feature based on percentage
    def style_function(feature):
        return {
            'fillColor': feature['properties']['color'],
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.5
        }

    # Add GeoJSON layer with hover functionality
    folium.GeoJson(
        parlamentarni_izbori,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=['hover_info'], 
            aliases=['Info: '],
            localize=True
        )
    ).add_to(m)

    # Add the colormap to the map
    colormap.add_to(m)

    # Ensure the folder exists
    os.makedirs(f"election", exist_ok=True)
    # Save the map as an HTML file
    file_name = f"election/interactive_map.html"
    m.save(file_name)

    # Display the map directly in the notebook
    display(m)




In [45]:
import matplotlib.pyplot as plt
from matplotlib.colors import to_hex, LinearSegmentedColormap

def create_winners_election_map(parlamentarni_izbori):
    # Identify columns ending with '%'
    percentage_columns = [col for col in parlamentarni_izbori.columns if col.endswith('%')]

    # Determine the highest percentage column and value for each Grad/Općina
    parlamentarni_izbori['highest_column'] = parlamentarni_izbori[percentage_columns].idxmax(axis=1)
    parlamentarni_izbori['highest_percentage'] = parlamentarni_izbori[percentage_columns].max(axis=1)

    # Assign unique base colors to parties
    unique_columns = parlamentarni_izbori['highest_column'].unique()
    base_colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'yellow']
    column_colors = {col: base_colors[i % len(base_colors)] for i, col in enumerate(unique_columns)}

    # Create a custom colormap for shading
    def get_shaded_color(base_color, percentage):
        # 50% maps to the base color; below 50% is lighter, above 50% is darker
        cmap = LinearSegmentedColormap.from_list(
            "custom_shade", 
            ["white", base_color, "black"], 
            N=256
        )
        return to_hex(cmap((percentage / 100)))

    # Apply shading based on percentages
    parlamentarni_izbori['shaded_color'] = parlamentarni_izbori.apply(
        lambda row: get_shaded_color(column_colors[row['highest_column']], row['highest_percentage']),
        axis=1
    )

    # Plot the map
    fig, ax = plt.subplots(figsize=(15, 15))
    parlamentarni_izbori.plot(
        color=parlamentarni_izbori['shaded_color'],
        edgecolor="black",
        linewidth=0.5,
        ax=ax
    )

    # Create a legend for each party
    legend_elements = []
    for col, base_color in column_colors.items():
        # Add the base color (50%) and gradients for 25% and 75%
        legend_elements.append(plt.Line2D(
            [0], [0], color='white', marker='None', label=f"\n{col}"
        ))
        for strength, shade in zip(
            [25, 50, 75], 
            [get_shaded_color(base_color, 25), get_shaded_color(base_color, 50), get_shaded_color(base_color, 75)]
        ):
            legend_elements.append(plt.Line2D(
                [0], [0], color=shade, marker='s', label=f"{strength}%", markersize=10, linestyle='None'
            ))

    # Add the legend to the map
    ax.legend(
        handles=legend_elements,
        title="Legend: Party & Percentage",
        loc='upper center',
        bbox_to_anchor=(0.5, -0.05),
        ncol=4,
        fontsize=8
    )

    # Add title and remove axis
    ax.set_title("Election Results by Grad/Općina", fontsize=18)
    ax.set_axis_off()

    # Save and display the map
    # Ensure the folder exists
    os.makedirs(f"election", exist_ok=True)
    plt.savefig("election/election_results_map.png", bbox_inches="tight", dpi=300)
    plt.show()


In [40]:
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colorbar import ColorbarBase


def create_election_map(parlamentarni_izbori, stranka):
    # Filter the dataframe for the specified stranka
    if stranka not in parlamentarni_izbori.columns:
        raise ValueError(f"The specified stranka '{stranka}' is not in the dataset.")
    
    # Ensure the percentage column for the stranka exists
    if not stranka.endswith('%'):
        stranka += '%'

    if stranka not in parlamentarni_izbori.columns:
        raise ValueError(f"The percentage column for '{stranka}' does not exist.")
    
    # Get the minimum and maximum percentage values for the stranka
    min_value = parlamentarni_izbori[stranka].min()
    max_value = parlamentarni_izbori[stranka].max()

    # Create a custom colormap for shading (from light blue to dark blue)
    colormap = LinearSegmentedColormap.from_list(
        "custom_spectrum", 
        ["lightblue", "blue", "darkblue"], 
        N=256
    )

    # Normalize values between the min and max
    norm = Normalize(vmin=min_value, vmax=max_value)

    # Apply shading based on the column's values
    parlamentarni_izbori['shaded_color'] = parlamentarni_izbori[stranka].apply(
        lambda percentage: colormap(norm(percentage)) if not pd.isna(percentage) else '#ffffff'
    )

    # Plot the map
    fig, ax = plt.subplots(figsize=(15, 15))
    parlamentarni_izbori.plot(
        color=parlamentarni_izbori['shaded_color'],
        edgecolor="black",
        linewidth=0.5,
        ax=ax
    )

    # Add a title
    ax.set_title(f"Election Results for {stranka} by Grad/Općina", fontsize=18)
    ax.set_axis_off()

    # Add a colorbar
    cax = fig.add_axes([0.25, 0.05, 0.5, 0.03])  # Position of the colorbar [left, bottom, width, height]
    colorbar = ColorbarBase(
        cax, cmap=colormap, norm=norm, orientation='horizontal'
    )
    colorbar.set_label(f"{stranka} Percentage (%)")
    colorbar.ax.tick_params(labelsize=10)

    # Save and display the map
    os.makedirs(f"election", exist_ok=True)
    plt.savefig(f"election/election_results_{stranka}.png", bbox_inches="tight", dpi=300)
    plt.show()

# Loop to generate maps for all parties with colorbars
def generate_maps_for_all_parties(parlamentarni_izbori):
    # Identify all columns that end with '%'
    percentage_columns = [col for col in parlamentarni_izbori.columns if col.endswith('%')]
    
    for stranka in percentage_columns:
        print(f"Generating map for: {stranka}")
        create_election_map(parlamentarni_izbori, stranka)


INTERAKTIVNA KARTA PARLAMENTARNIH IZBORA

In [ ]:
create_interactive_election_map(parlamentarni_izbori)

KARTA POBJEDNIČKIH STRANKI PARLAMENTARNIH IZBORA

In [ ]:
create_winners_election_map(parlamentarni_izbori)

KARTE REZULTATA STRANKI PARLAMENTARNIH IZBORA

In [ ]:
generate_maps_for_all_parties(parlamentarni_izbori)